<a href="https://colab.research.google.com/github/Voyager0001/Monitor-Distance-Sensitivity-Calculator/blob/main/Sensitivity_vs_Ratio_Plot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
DPI=1600
Sensitivity=1
cm360=25.9773
gameConstant = ((2.54*360)/(DPI*Sensitivity))/cm360
print(gameConstant)
Sensitivity=1
cm360=8.9773
gameConstant = ((2.54*360)/(DPI*Sensitivity))/cm360
print(gameConstant)

0.02199997690291139
0.06366056609448276


In [7]:
import math
import plotly.graph_objects as go
import numpy as np
import random # Import random module for colors

# --- User Defined Inputs ---

# Game 1 Settings
fov0_deg = 103.0          # Field of View for Game 1 (degrees)
game_sens0 = .187          # In-game sensitivity for Game 1
game_const0 = .069999971       # Sensitivity constant for Game 1 (e.g., m_yaw for Source engine)

# Game 2 Settings - Now a list of FOVs
fov1_deg_list = [70, 80, 90, 100, 106.260205] # List of FOVs for Game 2 (degrees)
# Assuming Game 2 constant is the same for all FOVs for now
game_const1 = 0.02199997690291139       # Sensitivity constant for Game 2

# General Settings
dpi = 1600                # Mouse DPI

# --- Constants and Conversions ---
# Convert Game 1 FOV from degrees to radians and take half
fov0_rad_half = fov0_deg * 0.5 * math.pi / 180

# Conversion factor from inches to cm
INCHES_TO_CM = 2.54
DEGREES_360 = 360

# --- Calculate sens0 (cm/360 for Game 1) ---
# This is calculated only once based on Game 1 settings
try:
    if dpi <= 0 or game_sens0 <= 0 or game_const0 <= 0:
        raise ValueError("DPI, Game 1 Sensitivity, and Game 1 Constant must be positive.")
    sens0_cm360 = (INCHES_TO_CM * DEGREES_360) / (dpi * game_sens0 * game_const0)
    print(f"Calculated sens0 (cm/360 for Game 1): {sens0_cm360:.4f} cm/360")
except ValueError as e:
    print(f"Error calculating initial cm/360: {e}")
    sens0_cm360 = None # Indicate error

# --- Generate Ratios ---
# Create ratios from 0 to 1 with a step of 0.01
# Replace 0 with a very small number as requested
ratios = np.arange(0.0, 1.01, 0.01) # Go up to 1.01 to include 1.0
ratios_plot = ratios.copy() # Keep a copy for plotting labels
ratios_plot[0] = 0.0 # Use 0 for the plot axis label
ratios[0] = 1e-9 # Use small number for calculation

# --- Initialize Plot ---
fig = go.Figure()

# --- Calculate Sensitivities and Plot for each Game 2 FOV ---
if sens0_cm360 is not None: # Proceed only if initial calculation was successful
    # Loop through each FOV defined for Game 2
    for fov1_deg in fov1_deg_list:
        print(f"\nCalculating for Game 2 FOV: {fov1_deg}°")
        # Convert current Game 2 FOV to radians and take half
        fov1_rad_half = fov1_deg * 0.5 * math.pi / 180

        game_sensitivities_1 = [] # Store calculated sensitivities for this specific FOV

        # Calculate sensitivities for the current FOV across all ratios
        for ratio in ratios:
            sens1_cm360 = None # Initialize target cm/360 for this ratio
            game_sens1 = None  # Initialize target game sensitivity for this ratio

            try:
                # Calculate alpha0 and alpha1 using the monitor distance formulas
                tan_fov0 = math.tan(fov0_rad_half)
                tan_fov1 = math.tan(fov1_rad_half) # Use the current FOV's tan value

                alpha0 = math.atan(ratio * tan_fov0)
                alpha1 = math.atan(ratio * tan_fov1)

                # Calculate sens1 (cm/360 equivalent for Game 2)
                if alpha1 != 0:
                    sens1_cm360 = sens0_cm360 * alpha0 / alpha1
                elif alpha0 == 0: # If ratio is near zero, both alphas are near zero
                     sens1_cm360 = sens0_cm360 * tan_fov0 / tan_fov1 # Use the limit
                else:
                     sens1_cm360 = float('inf')

                # Calculate game_sens1 (in-game sensitivity for Game 2)
                if sens1_cm360 is not None and sens1_cm360 > 1e-9 and dpi > 0 and game_const1 > 0:
                    game_sens1 = (INCHES_TO_CM * DEGREES_360) / (dpi * sens1_cm360 * game_const1)
                elif sens1_cm360 is not None and sens1_cm360 <= 1e-9:
                    game_sens1 = float('inf') # Very low cm/360 means very high in-game sens
                else:
                     game_sens1 = None

            except ValueError as e:
                print(f"  Math error for ratio {ratio}: {e}")
                # Keep game_sens1 as None

            game_sensitivities_1.append(game_sens1)

        # Filter out None or infinite values for plotting
        valid_indices = [i for i, s in enumerate(game_sensitivities_1) if s is not None and np.isfinite(s)]
        valid_ratios_for_plot = [ratios_plot[i] for i in valid_indices] # Use ratios_plot for x-axis
        valid_game_sensitivities = [game_sensitivities_1[i] for i in valid_indices]

        # Generate a random color for this FOV's line
        rand_color = f'rgb({random.randint(0,255)}, {random.randint(0,255)}, {random.randint(0,255)})'

        # Add the trace for the current FOV to the plot
        if valid_game_sensitivities: # Only plot if there's valid data
            fig.add_trace(go.Scatter(
                x=valid_ratios_for_plot,
                y=valid_game_sensitivities,
                mode='lines', # Use lines only for potentially smoother look with multiple lines
                name=f'Game 2 Sens (FOV {fov1_deg}°)', # Label line with its FOV
                line=dict(color=rand_color) # Assign the random color
            ))

            # Print sample values for this FOV
            print(f"  Sample Values (Ratio: Game 2 Sens @ {fov1_deg}° FOV):")
            for i in range(min(5, len(valid_ratios_for_plot))):
                 print(f"    {valid_ratios_for_plot[i]:.2f}: {valid_game_sensitivities[i]:.6f}")
            if len(valid_ratios_for_plot) > 0:
                 print(f"    {valid_ratios_for_plot[-1]:.2f}: {valid_game_sensitivities[-1]:.6f}")
        else:
             print(f"  No valid data to plot for FOV {fov1_deg}°.")


# --- Finalize and Show Plot ---
if fig.data: # Check if any traces were actually added
    # Update layout for titles and labels
    fig.update_layout(
        title=f'Calculated Game 2 In-Game Sensitivity vs. Monitor Distance Ratio (Multiple FOVs)',
        xaxis_title='Monitor Distance Ratio',
        yaxis_title='Calculated Game 2 In-Game Sensitivity',
        xaxis=dict(range=[0, 1]), # Set x-axis range from 0 to 1
        hovermode='x unified' # Show hover info for the x-value across all lines
    )
    fig.show()
else:
    print("\nCould not generate plot. Initial calculation failed or no valid data for any FOV.")



Calculated sens0 (cm/360 for Game 1): 43.6593 cm/360

Calculating for Game 2 FOV: 70°
  Sample Values (Ratio: Game 2 Sens @ 70° FOV):
    0.00: 0.331397
    0.01: 0.331410
    0.02: 0.331446
    0.03: 0.331506
    0.04: 0.331590
    1.00: 0.404369

Calculating for Game 2 FOV: 80°
  Sample Values (Ratio: Game 2 Sens @ 80° FOV):
    0.00: 0.397133
    0.01: 0.397145
    0.02: 0.397179
    0.03: 0.397237
    0.04: 0.397318
    1.00: 0.462136

Calculating for Game 2 FOV: 90°
  Sample Values (Ratio: Game 2 Sens @ 90° FOV):
    0.00: 0.473285
    0.01: 0.473294
    0.02: 0.473321
    0.03: 0.473367
    0.04: 0.473431
    1.00: 0.519903

Calculating for Game 2 FOV: 100°
  Sample Values (Ratio: Game 2 Sens @ 100° FOV):
    0.00: 0.564039
    0.01: 0.564042
    0.02: 0.564051
    0.03: 0.564066
    0.04: 0.564087
    1.00: 0.577670

Calculating for Game 2 FOV: 106.260205°
  Sample Values (Ratio: Game 2 Sens @ 106.260205° FOV):
    0.00: 0.631046
    0.01: 0.631042
    0.02: 0.631030
    0.03: 0